In [ ]:
# This cell tells python to use the local version of CVToolkit instead of the installed one.
# Use for testing purposes until branch feature/BCV-970-oor-metrics is merged.
import sys
import os

module_path = os.path.abspath(os.path.join("../../CVToolkit"))
if module_path not in sys.path:
    sys.path.insert(0, module_path)

In [ ]:
import cv2
import pathlib
import sys

sys.path.append("../..")

from objectherkenning_openbare_ruimte.performance_evaluation_pipeline.metrics import tba_calculator
from objectherkenning_openbare_ruimte.data_minimalisation_pipeline.source import blurring_tools

In [ ]:
from enum import Enum

class Scenario(Enum):
    A = True, False, False, True
    B = True, True, False, False
    C = True, False, True, False, True

    def __init__(self, blur_inside: bool, blur_outside: bool, crop: bool, draw_box: bool, fill_bg: bool = False):
        self.blur_inside = blur_inside
        self.blur_outside = blur_outside
        self.crop = crop
        self.draw_box = draw_box
        self.fill_bg = fill_bg

In [ ]:
data_folder = "../../datasets/oor/data-minimalisation"
label_folder = "predictions"
output_folder = "../../datasets/oor/data-minimalisation/blurred"

blur_kernel_size_outside = 55
blur_kernel_size_inside = 165
blur_outside_padding = 25
crop_padding = 25

scenario = Scenario.A

In [ ]:
pathlib.Path(output_folder).mkdir(parents=True, exist_ok=True)

for img_file in pathlib.Path(os.path.join(data_folder, "images")).glob("*.png"):
    filename = img_file.stem
    print(f"=== {filename} ===")
    label_file = os.path.join(data_folder, label_folder, filename + ".txt")

    image_raw = cv2.imread(img_file.as_posix(), cv2.IMREAD_COLOR)

    if not os.path.isfile(label_file):
        print("Nothing to blur!")
    else:
        with open(label_file, "r") as file:
            lines = sorted(file.readlines())
            for scenario in Scenario:
                print(f"--- Scenario {scenario.name} ---")
                image = image_raw.copy()
                for line in lines:
                    yolo_annotation = line.strip()
                    class_id = int(yolo_annotation.split(sep=" ", maxsplit=1)[0])
                    if scenario.blur_inside and (class_id in (0, 1)):
                        image = blurring_tools.blur_inside_yolo_box(
                            image, yolo_annotation, blur_kernel_size_inside)
                    if scenario.blur_outside and (class_id in (2,)):
                        image = blurring_tools.blur_outside_yolo_box(
                            image, yolo_annotation, blur_kernel_size_outside, blur_outside_padding)
                    if scenario.crop and (class_id in (2,)):
                        image = blurring_tools.crop_outside_yolo_box(image, yolo_annotation, crop_padding, scenario.fill_bg)
                    if scenario.draw_box and (class_id in (2,)):
                        image = blurring_tools.draw_yolo_box(image, yolo_annotation)
                
                out_path = os.path.join(output_folder, f"{filename}_scen_{scenario.name}.jpg")
                cv2.imwrite(out_path, image)

In [ ]:
from PIL import Image
from IPython.display import display

display(Image.fromarray(image[:, :, ::-1]))

In [ ]:
cv2.imwrite(f"../../datasets/oor/data-minimalisation/scenario_{scenario.name}.jpg", image)

## Collect sample data

In [ ]:
# Find all images with containers

import os
import pandas as pd

from typing import List

def get_labels(label_file: str) -> List[int]:
    labels: List[int] = [0]*5
    if not os.path.isfile(label_file):
        return labels
    with open(label_file, "r") as file:
        lines = file.readlines()
        for line in lines:
            label = int(line.split(sep=" ")[0])
            labels[label] += 1
    return labels

data_folder = "../../datasets/oor/first-train-oor-nc5"

data = {
    "filename": [],
    "n_person": [],
    "n_license_plate": [],
    "n_container": [],
}

images = list(pathlib.Path(os.path.join(data_folder, "images", "val")).glob("*.png"))

for img in images:
    filename = img.stem
    label_file = os.path.join(data_folder, "labels", "val", filename + ".txt")
    labels = get_labels(label_file)
    data["filename"].append(filename)
    data["n_person"].append(labels[0])
    data["n_license_plate"].append(labels[1])
    data["n_container"].append(labels[2])

df = pd.DataFrame(data=data)

In [ ]:
import shutil

filenames = df.query("n_container > 0 & (n_license_plate > 0 | n_person > 0)")["filename"].to_list()

target_dir = "../../datasets/oor/data-minimalisation"

pathlib.Path(os.path.join(target_dir, "images")).mkdir(parents=True, exist_ok=True)
pathlib.Path(os.path.join(target_dir, "labels")).mkdir(parents=True, exist_ok=True)

for file in filenames:
    img_src = os.path.join(data_folder, "images", "val", file + ".png")
    img_dst = os.path.join(target_dir, "images", file + ".png")
    shutil.copyfile(img_src, img_dst)
    lab_src = os.path.join(data_folder, "labels", "val", file + ".txt")
    lab_dst = os.path.join(target_dir, "labels", file + ".txt")
    shutil.copyfile(lab_src, lab_dst)